In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import os
import yaml
import shutil
from collections import defaultdict

matplotlib.style.use('seaborn-notebook')
matplotlib.pyplot.rcParams['figure.figsize'] = (16, 10)
matplotlib.pyplot.rcParams['font.family'] = 'sans-serif'

%matplotlib inline
%load_ext autoreload
%autoreload 2

# Task1

In [2]:
%time
exp_dir = "../exps/task1/two_headed_best/"
dev_outputs = defaultdict(list)

def load_exps(exp_dir):
    experiments = []
    for subdir in os.scandir(exp_dir):
        dev_fn = os.path.join(subdir.path, "dev.out")
        if not os.path.exists(dev_fn):
            print("No dev.out: {}".format(subdir.path))
            continue
        config_fn = os.path.join(subdir.path, "config.yaml")
        with open(config_fn) as f:
            config = yaml.load(f)
        config['config_language'] = config['language']
        config['language'] = '-'.join(config['dev_file'].split('/')[-1].split('-')[:-1])
        config['train_size'] = config['train_file'].split('/')[-1].split('-')[-1]
        exp_d = config
        dev_output = pd.read_table(dev_fn, names=["lemma", "inflected", "tags"])
        if len(dev_output['inflected']) == 0:
            print(subdir.path)
        
        dev_outputs[(config['language'], config['train_size'])].append(dev_output)
        
        dev_acc_fn = os.path.join(subdir.path, "dev.word_accuracy")
        if not os.path.exists(dev_acc_fn):
            print("Dev accuracy file does not exist in dir: {}".format(subdir.path))
        else:
            with open(dev_acc_fn) as f:
                exp_d['dev_acc'] = float(f.read())
        experiments.append(exp_d)
    experiments = pd.DataFrame(experiments)
    return experiments
        
experiments = load_exps(exp_dir)
sum(len(v) for v in dev_outputs.values())

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 3.1 µs


4365

In [3]:
gold = {}

for fn in os.scandir("../data/conll2018/task1/all"):
    if not fn.path.endswith('-dev'):
        continue
    language = '-'.join(fn.name.split('-')[:-1])
    with open(fn.path) as f:
        inflected = [l.strip().split('\t')[1] for l in f]
        gold[language] = pd.Series(inflected)

for fn in os.scandir("../data/conll2018/task1/surprise"):
    if not fn.path.endswith('-dev'):
        continue
    language = '-'.join(fn.name.split('-')[:-1])
    with open(fn.path) as f:
        inflected = [l.strip().split('\t')[1] for l in f]
        gold[language] = pd.Series(inflected)

In [4]:
experiments.groupby('train_size').dev_acc.describe()

,count,mean,std,min,25%,50%,75%,max
train_size,,,,,,,,
high,1290.0,0.907692,0.102617,0.001,0.866,0.9380,0.97200,1.00
low,1545.0,0.012261,0.040310,0.000,0.000,0.0000,0.00000,0.48
medium,1530.0,0.467520,0.305044,0.000,0.196,0.4465,0.75175,1.00


In [5]:
merged = {}
for (lang, size), outputs in dev_outputs.items():
    output = pd.concat(outputs, axis=1)
    merged[(lang, size)] = pd.concat(outputs, axis=1)['inflected']
    
merged[('latin', 'high')].head()

,inflected,inflected,inflected,inflected,inflected,inflected,inflected,inflected,inflected,inflected,inflected,inflected,inflected,inflected,inflected
0,ovillī,ovillī,ovillī,ovillī,ovillī,ovillī,ovillī,ovillī,ovillī,obellī,ovillī,ovillī,ovillī,ovillī,ovillī
1,trȳgōnibus,trȳgōnibus,trygōnibus,trygōnibus,trȳgōnibus,trȳgōnibus,trȳgōnibus,trȳginibus,trȳgōnibus,torgīs,trȳgōnibus,trygonibus,trȳgīs,trȳgōnibus,trygōnibus
2,largīvissētis,largīvissētis,largīvissētis,largīvissētis,largīvissētis,largissētis,largīvissētis,largīvissētis,largīvissētis,largīvissētis,largīvissētis,largīvissētis,largīvissētis,largīvissētis,largīvissētis
3,compensāverit,compensāverit,compenserit,compensāverit,compensāverit,compenserit,compenserit,compensāverit,compensāverit,compensāverit,compensāverit,compensāverit,compensāverit,compensāverit,compenserit
4,comperīrer,comperiārer,comperīrer,comperiārer,comperīrer,comperīrer,comperierer,comperīrer,comperiēbar,comperīrer,comperīrer,comperiārer,comperīrer,comperīrer,comperīrer


In [6]:
results = []

for (lang, size), outputs in merged.items():
    d = {'language': lang, 'train_size': size}
    for i in range(outputs.shape[1]):
        acc = (outputs.iloc[:, i] == gold[lang]).mean()
        d[i] = acc
    d['majority'] = (outputs.mode(axis=1).iloc[:, 0] == gold[lang]).mean()
    results.append(d)

In [7]:
dev_results = pd.DataFrame(results)

In [8]:
dev_results['majority_wins'] = dev_results[dev_results.columns[2:-1]].max(axis=1) < dev_results.majority

In [9]:
dev_results[dev_results['train_size']=='low'].majority_wins.value_counts()
dev_results.majority_wins.value_counts()

False    227
True      64
Name: majority_wins, dtype: int64

### Save best configs

In [28]:
outdir = "../config/task1/highest_dev_acc"
keep = ['batch_size', 'commit_hash', 'dataset_class', 'dropout', 'early_stopping_window', 'epochs',
 'inflected_embedding_size', 'inflected_hidden_size', 'inflected_num_layers', 
 'lemma_embedding_size', 'lemma_hidden_size', 'lemma_num_layers', 'model', 
 'numpy_random_seed', 'optimizer', 'share_vocab', 
 'tag_embedding_size', 'tag_hidden_size', 'tag_num_layers', 'train_file', 'dev_file',
 'save_min_epoch', 'torch_random_seed']

for (lang, train_size), idx in experiments.groupby(['language', 'train_size']).dev_acc.idxmax().iteritems():
    out_fn = os.path.join(outdir, "{}_{}.yaml".format(lang, train_size))
    in_config = experiments.loc[idx].experiment_dir + "/config.yaml"
    with open(in_config) as f:
        cfg = yaml.load(f)
    cfg = {k: v for k, v in cfg.items() if k in keep}
    cfg['train_file'] = 'data/conll2018' + cfg['train_file'].split('conll2018')[1]
    cfg['dev_file'] = 'data/conll2018' + cfg['dev_file'].split('conll2018')[1]
    with open(out_fn, 'w') as f:
        yaml.dump(cfg, f, default_flow_style=False)

# Submission 1

Model with the highest dev acc, no majority voting

In [35]:
output_dir = "../submissions/task1/01/task1"
for (lang, train_size), idx in experiments.groupby(['language', 'train_size']).dev_acc.idxmax().iteritems():
    exp = experiments.loc[idx]
    test_output = os.path.join(exp.experiment_dir, "test.out")
    submission_path = os.path.join(output_dir, "{}-{}-out".format(lang, train_size))
    shutil.copy2(test_output, submission_path)

# Submission 2

Simple majority vote

In [36]:
output_dir = "../submissions/task1/02/task1"
for (lang, train_size), group in experiments.groupby(['language', 'train_size']).groups.items():
    all_outputs = None
    for i, idx in enumerate(group):
        exp = experiments.loc[idx]
        test_fn = os.path.join(exp.experiment_dir, "test.out")
        test_output = pd.read_table(test_fn, names=['lemma', i, 'tags'])
        if all_outputs is None:
            all_outputs = test_output[[i]]
        else:
            all_outputs = pd.concat((all_outputs, test_output[[i]]), axis=1)
    majority = pd.concat((test_output[['lemma']], all_outputs.mode(axis=1)[0], test_output[['tags']]), axis=1)
    output_fn = os.path.join(output_dir, "{}-{}-out".format(lang, train_size))
    majority.to_csv(output_fn, index=False, header=False, sep="\t", na_rep='nan')

# Submission 3

batch size = 16

In [37]:
%time
exp_dir = "../exps/task1/two_headed_best/"

def load_exps(exp_dir):
    experiments = []
    for subdir in os.scandir(exp_dir):
        dev_fn = os.path.join(subdir.path, "dev.batch16.out")
        if not os.path.exists(dev_fn):
            print("No dev.out: {}".format(subdir.path))
            continue
        config_fn = os.path.join(subdir.path, "config.yaml")
        with open(config_fn) as f:
            config = yaml.load(f)
        config['config_language'] = config['language']
        config['language'] = '-'.join(config['dev_file'].split('/')[-1].split('-')[:-1])
        config['train_size'] = config['train_file'].split('/')[-1].split('-')[-1]
        exp_d = config
        dev_acc_fn = os.path.join(subdir.path, "dev.batch16.word_accuracy")
        if not os.path.exists(dev_acc_fn):
            print("Dev accuracy file does not exist in dir: {}".format(subdir.path))
        else:
            with open(dev_acc_fn) as f:
                exp_d['dev_acc'] = float(f.read())
        experiments.append(exp_d)
    experiments = pd.DataFrame(experiments)
    return experiments
        
experiments = load_exps(exp_dir)
sum(len(v) for v in dev_outputs.values())

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 3.34 µs


4365

In [38]:
output_dir = "../submissions/task1/03/task1"
for (lang, train_size), idx in experiments.groupby(['language', 'train_size']).dev_acc.idxmax().iteritems():
    exp = experiments.loc[idx]
    test_output = os.path.join(exp.experiment_dir, "test.out")
    submission_path = os.path.join(output_dir, "{}-{}-out".format(lang, train_size))
    shutil.copy2(test_output, submission_path)

# Task2 - Track 1

In [29]:
exp_dirs = ["../exps/task2/track1/final", "../exps/task2/track1/default"]
task2_track1 = []

for exp_dir in exp_dirs:
    for subdir in os.scandir(exp_dir):
        config_fn = os.path.join(subdir.path, "config.yaml")
        with open(config_fn) as f:
            config = yaml.load(f)
        exp_d = config
        exp_d['language'] = config['train_file'].split('/')[-1].split('-')[0]
        exp_d['train_size'] = config['train_file'].split('/')[-1].split('-')[2]
        dev_acc_fn = os.path.join(subdir.path, "dev.word_accuracy")
        if not os.path.exists(dev_acc_fn):
            print("Dev accuracy file does not exist in {}".format(subdir.path))
        else:
            with open(dev_acc_fn) as f:
                exp_d['dev_acc'] = float(f.read())
        test_out_fn = os.path.join(subdir.path, "test.out")
        if not os.path.exists(test_out_fn):
            print("Test output file does not exist in {}".format(subdir.path))
        else:
            exp_d['test_output_path'] = test_out_fn
            
        task2_track1.append(exp_d)
        
task2_track1 = pd.DataFrame(task2_track1)

Test output file does not exist in ../exps/task2/track1/default/0162
Test output file does not exist in ../exps/task2/track1/default/0163
Test output file does not exist in ../exps/task2/track1/default/0164
Test output file does not exist in ../exps/task2/track1/default/0165
Test output file does not exist in ../exps/task2/track1/default/0166


In [56]:
task2_track1.groupby(['language', 'train_size']).dev_acc.agg(['mean', 'max', 'std', 'size'])

mean       max       std  size
language train_size                                    
de       high        0.593093  0.741742  0.199972     8
         low         0.136470  0.270270  0.124969     9
         medium      0.420087  0.624625  0.240055     9
en       high        0.617794  0.732558  0.190157     9
         low         0.332353  0.545309  0.247739     9
         medium      0.478437  0.610666  0.172554     9
es       high        0.497256  0.551146  0.069132     9
         low         0.152361  0.229718  0.078173     9
         medium      0.308299  0.394621  0.117447     9
fi       high        0.394072  0.462520  0.057271     8
         low         0.044989  0.095162  0.036843     8
         medium      0.165670  0.210526  0.038634     8
fr       high        0.595921  0.671299  0.080760     8
         low         0.107035  0.244713  0.104946     8
         medium      0.376056  0.464653  0.083996    44
ru       high        0.624612  0.692633  0.088557     7
         low         0.127717  0.209145  0.094539     7
         medium      0.281118  0.403895  0.152874     7
sv       high        0.594822  0.691099  0.113861     6
         low         0.118674  0.280977  0.122318     6
         medium      0.327807  0.390925  0.068089     6

In [60]:
t = task2_track1
t[(t.language=='sv') & (t.train_size=='high')]

,batch_size,cell_type,char_embedding_size,commit_hash,context_hidden_size,context_num_layers,dataset_class,decoder_hidden_size,decoder_num_layers,defaults,...,torch_random_seed,toy_eval,train_file,train_size,use_eos,use_step,vocab_path_src,vocab_path_tgt,word_hidden_size,word_num_layers
18,32,LSTM,30,818c7f814ba005b6165d3cca297ee3e4f58bc354,256,1,SIGMORPOHTask2Track1Dataset,64,1,"{'cell_type': 'LSTM', 'dataset_class': 'Labele...",...,788145932,None,/mnt/permanent/home/judit/projects/sigmorphon2...,high,True,False,/mnt/permanent/home/judit/projects/sigmorphon2...,/mnt/permanent/home/judit/projects/sigmorphon2...,64,2
52,16,LSTM,40,c5c59d5ebef7b8d35aa5f146653c5d93ce8c8230,128,1,SIGMORPOHTask2Track1Dataset,128,1,"{'cell_type': 'LSTM', 'dataset_class': 'Labele...",...,1839500317,None,/mnt/permanent/home/judit/projects/sigmorphon2...,high,True,False,/mnt/permanent/home/judit/projects/sigmorphon2...,/mnt/permanent/home/judit/projects/sigmorphon2...,128,1
74,16,LSTM,40,2cccbc9dae4ab04eac491a3fe3fe02d7f1ed581b,128,1,SIGMORPOHTask2Track1Dataset,128,1,"{'cell_type': 'LSTM', 'dataset_class': 'Labele...",...,1818896580,None,/mnt/permanent/home/judit/projects/sigmorphon2...,high,True,False,/mnt/permanent/home/judit/projects/sigmorphon2...,/mnt/permanent/home/judit/projects/sigmorphon2...,128,1
94,16,LSTM,40,be844cc98b151054ad0ef5b1a2848d14f8d07c3b,256,1,SIGMORPOHTask2Track1Dataset,256,1,"{'cell_type': 'LSTM', 'dataset_class': 'Labele...",...,604302163,None,/mnt/permanent/home/judit/projects/sigmorphon2...,high,True,False,/mnt/permanent/home/judit/projects/sigmorphon2...,/mnt/permanent/home/judit/projects/sigmorphon2...,256,1
118,32,LSTM,40,2169238bc622c22c5162642080af87762526d894,256,1,SIGMORPOHTask2Track1Dataset,256,1,"{'cell_type': 'LSTM', 'dataset_class': 'Labele...",...,747061712,None,/mnt/permanent/home/judit/projects/sigmorphon2...,high,True,False,/mnt/permanent/home/judit/projects/sigmorphon2...,/mnt/permanent/home/judit/projects/sigmorphon2...,256,1
138,32,LSTM,40,a5c83b8eccd0becea5faab3e15ddd45accca7944,256,1,SIGMORPOHTask2Track1Dataset,256,1,"{'cell_type': 'LSTM', 'dataset_class': 'Labele...",...,1962390220,None,/mnt/permanent/home/judit/projects/sigmorphon2...,high,True,False,/mnt/permanent/home/judit/projects/sigmorphon2...,/mnt/permanent/home/judit/projects/sigmorphon2...,256,1


## Save best configs

In [41]:
outdir = "../config/task2/track1/highest_dev_acc"


keep = ['batch_size', 'char_embedding_size', 'commit_hash', 'context_hidden_size', 'context_num_layers',
        'dataset_class', 'decoder_hidden_size', 'decoder_num_layers', 
        'dev_file', 'dropout', 'early_stopping_window', 'epochs', 
        'include_same_forms_ratio', 'lemma_hidden_size', 'lemma_num_layers',
        'min_epochs', 'model', 'numpy_random_seed', 'optimizer', 
        'share_context_encoder', 'share_embedding', 'share_vocab', 
        'tag_embedding_size', 'tag_hidden_size', 'tag_num_layers',
        'save_min_epoch',
        'torch_random_seed', 'train_file', 'word_hidden_size', 'word_num_layers']

for (language, train_size), idx in task2_track1.groupby(['language', 'train_size']).dev_acc.idxmax().iteritems():
    out_fn = os.path.join(outdir, "{}_{}.yaml".format(language, train_size))
    in_config = task2_track1.loc[idx].experiment_dir + "/config.yaml"
    with open(in_config) as f:
        cfg = yaml.load(f)
    cfg = {k: v for k, v in cfg.items() if k in keep}
    cfg['train_file'] = 'data/conll2018' + cfg['train_file'].split('conll2018')[1]
    cfg['dev_file'] = 'data/conll2018' + cfg['dev_file'].split('conll2018')[1]
    with open(out_fn, 'w') as f:
        yaml.dump(cfg, f, default_flow_style=False)

## Submission - there is only one

choose the one with the highest dev acc

In [ ]:
submission_dir = "../submissions/task2/01/task2"
for (language, train_size), idx in task2_track1.groupby(['language', 'train_size']).dev_acc.idxmax().iteritems():
    experiment = task2_track1.loc[idx]
    target_name = "{}-1-{}-out".format(language, train_size)
    target_fn = os.path.join(submission_dir, target_name)
    shutil.copy2(experiment.test_output_path, target_fn)

In [42]:
exp_dirs = ["../exps/task2/track2/default/"]
task2_track2 = []

for exp_dir in exp_dirs:
    for subdir in os.scandir(exp_dir):
        config_fn = os.path.join(subdir.path, "config.yaml")
        with open(config_fn) as f:
            config = yaml.load(f)
        exp_d = config
        exp_d['language'] = config['train_file'].split('/')[-1].split('-')[0]
        exp_d['train_size'] = config['train_file'].split('/')[-1].split('-')[2]
        dev_acc_fn = os.path.join(subdir.path, "dev.word_accuracy")
        if not os.path.exists(dev_acc_fn):
            print("Dev accuracy file does not exist in {}".format(subdir.path))
        else:
            with open(dev_acc_fn) as f:
                exp_d['dev_acc'] = float(f.read())
        test_out_fn = os.path.join(subdir.path, "test.out")
        if not os.path.exists(test_out_fn):
            print("Test output file does not exist in {}".format(subdir.path))
        else:
            exp_d['test_output_path'] = test_out_fn
            
        task2_track2.append(exp_d)
        
task2_track2 = pd.DataFrame(task2_track2)

In [43]:
task2_track2.groupby(['language', 'train_size']).dev_acc.max().to_frame()

dev_acc
language train_size          
de       high        0.594595
         low         0.264264
         medium      0.531532
en       high        0.652366
         low         0.511628
         medium      0.563352
es       high        0.380511
         low         0.245150
         medium      0.284832
fi       high        0.287613
         low         0.057948
         medium      0.152578
fr       high        0.477341
         low         0.083384
         medium      0.235045
ru       high        0.635055
         low         0.176969
         medium      0.358171
sv       high        0.595113
         low         0.202443
         medium      0.375218

### Save best configs

In [44]:
outdir = "../config/task2/track2/highest_dev_acc"


keep = ['batch_size', 'char_embedding_size', 'commit_hash', 'context_hidden_size', 'context_num_layers',
        'dataset_class', 'decoder_hidden_size', 'decoder_num_layers', 
        'dev_file', 'dropout', 'early_stopping_window', 'epochs', 
        'include_same_forms_ratio', 'lemma_hidden_size', 'lemma_num_layers',
        'min_epochs', 'model', 'numpy_random_seed', 'optimizer', 
        'share_context_encoder', 'share_embedding', 'share_vocab', 
        'tag_embedding_size', 'tag_hidden_size', 'tag_num_layers',
        'save_min_epoch',
        'torch_random_seed', 'train_file', 'word_hidden_size', 'word_num_layers']

for (language, train_size), idx in task2_track2.groupby(['language', 'train_size']).dev_acc.idxmax().iteritems():
    out_fn = os.path.join(outdir, "{}_{}.yaml".format(language, train_size))
    in_config = task2_track2.loc[idx].experiment_dir + "/config.yaml"
    with open(in_config) as f:
        cfg = yaml.load(f)
    cfg = {k: v for k, v in cfg.items() if k in keep}
    cfg['train_file'] = 'data/conll2018' + cfg['train_file'].split('conll2018')[1]
    cfg['dev_file'] = 'data/conll2018' + cfg['dev_file'].split('conll2018')[1]
    with open(out_fn, 'w') as f:
        yaml.dump(cfg, f, default_flow_style=False)

In [ ]:
submission_dir = "../submissions/task2/01/task2"
for (language, train_size), idx in task2_track2.groupby(['language', 'train_size']).dev_acc.idxmax().iteritems():
    experiment = task2_track2.loc[idx]
    target_name = "{}-2-{}-out".format(language, train_size)
    target_fn = os.path.join(submission_dir, target_name)
    shutil.copy2(experiment.test_output_path, target_fn)

# Sanity checks

## Task1

1. Is every submission file the same length as the input?
2. Does every line have 3 fields?
3. Do the inputs match?

In [39]:
test_dir = "../data/conll2018/task1/all"
submission_dir = "../submissions/task1"

for submission_no in os.listdir(submission_dir):
    for subm_file in os.scandir(os.path.join(submission_dir, submission_no, 'task1')):
        language = '-'.join(subm_file.name.split('-')[:-2])
        test_fn = os.path.join(test_dir, '{}-covered-test'.format(language))
        if not os.path.exists(test_fn):
            test_fn = os.path.join(test_dir, "..", "surprise", '{}-covered-test'.format(language))
        with open(test_fn) as input_f, open(subm_file) as output_f:
            for inp_line in input_f:
                outp_line = next(output_f)
                infd = inp_line.strip().split('\t')
                outfd = outp_line.strip().split('\t')
                try:
                    assert len(outfd) == 3
                    assert infd[0] == outfd[0]
                    assert infd[-1] == outfd[-1]
                except AssertionError:
                    print(submission_no, subm_file.path, infd, outfd)

## Task2

In [ ]:
test_dir = "../data/conll2018/task2/testsets/"
submission_dir = "../submissions/task2"

for submission_no in os.listdir(submission_dir):
    for subm_file in os.scandir(os.path.join(submission_dir, submission_no, 'task2')):
        language = subm_file.name.split('-')[0]
        track = subm_file.name.split('-')[1]
        test_fn = os.path.join(test_dir, '{}-track{}-covered'.format(language, track))
        with open(test_fn) as input_f, open(subm_file) as output_f:
            for inp_line in input_f:
                outp_line = next(output_f)
                if not inp_line.strip():
                    assert not outp_line.strip()
                    continue
                infd = inp_line.strip().split('\t')
                outfd = outp_line.strip().split('\t')
                if track == '1':
                    try:
                        assert len(outfd) == 3
                        assert infd[2] == outfd[2]
                    except AssertionError:
                        print(inp_line)
                        print(outp_line)
                        print(submission_no, subm_file.path, infd, outfd)
                elif track == '2':
                    pass
                else:
                    raise ValueError("unknown track: {}".format(track))

# Create submission archives

## Task1

In [46]:
%%bash

old_pwd=$(pwd)

for sub in $( ls ../submissions/task1); do
    tar_name="BME-HAS-$sub-1.tgz"
    echo $sub
    cd ../submissions/task1/$sub
    mkdir -p arabic_slovene_fixed
    mkdir -p arabic_slovene_fixed/task1
    cp task1/arabic* arabic_slovene_fixed/task1
    cp task1/slovene* arabic_slovene_fixed/task1
    cd arabic_slovene_fixed
    tar czf $tar_name task1
    mv $tar_name ../../../tgz/arabic_slovene_fixed/
    cd $old_pwd
done

01
02
03


In [ ]:
%%bash

for sub in $( ls ../submissions/task1); do
    tar_name="BME-HAS-$sub-1.tgz"
    cd ../submissions/task1/$sub
    tar czf $tar_name task1
    mv $tar_name ../../tgz
    cd -
done

In [ ]:
%%bash

for sub in $( ls ../submissions/task2); do
    tar_name="BME-HAS-$sub-2.tgz"
    cd ../submissions/task2/$sub
    tar czf $tar_name task2
    mv $tar_name ../../tgz
    cd -
done